# MemoryPC and CSL Computer - Configuration

Get the best price for your PC.

## How To

Create your configuration on MemoryPC and CSL Computer.

Copy the name of each component you want in your PC configuration and add them to `MEMORYPC_CONFIGURATIONS` and `CSL_COMPUTER_CONFIGURATIONS` dictionnaries.

If you want "backup" components (like AMD first choice and Intel 2nd choice), you can make a list for each component type: list of CPUs, motherboard, ...

The keys used in the dictionnary do not matter.

You can also adpat the search pages to match your configuration (like exclude AMD or intel configurations)

In [1]:
import sys
!{sys.executable} -m pip install selenium webdriver-manager wakepy requests

import tempfile
import subprocess
import requests
import time
import shutil
import re
from selenium import webdriver
from selenium.webdriver.remote.webdriver import WebDriver
from selenium.webdriver.chrome.webdriver import WebDriver as BraveDriver
from selenium.webdriver.chrome.service import Service as BraveService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.core.os_manager import ChromeType
from selenium.webdriver.remote.webelement import WebElement
from typing import List, Dict, Optional, Union
import subprocess
import time
import re
import requests
import shutil
from wakepy import keep
from abc import ABC, abstractmethod
from concurrent.futures import ThreadPoolExecutor, as_completed
from contextlib import contextmanager
import shutil

In [2]:
# Auto-detect Chromium version from Brave
def get_chromium_version_from_brave(brave_path, debug_port = 9333):
    """
    Get the Chromium version from Brave by launching it and checking brave://version
    This is needed because webdriver_manager needs the Chromium version, not the Brave version.
    """
    
    # Create a temporary user data directory and debugging port
    temp_dir = tempfile.mkdtemp()
    
    try:
        # Launch Brave in headless mode with remote debugging
        process = subprocess.Popen([
            brave_path,
            f'--user-data-dir={temp_dir}',
            f'--remote-debugging-port={debug_port}',
            '--headless=new',
            '--no-sandbox',
            '--disable-dev-shm-usage'
        ], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Wait a bit for Brave to start
        time.sleep(2)
        
        # Connect to the remote debugging interface
        response = requests.get(f'http://localhost:{debug_port}/json/version')
        
        if response.status_code == 200:
            version_info = response.json()
            # Extract Chrome version (format: "Chrome/144.0.7559.59")
            browser_info = version_info.get('Browser', '')
            browser_version = re.search(r'Chrome/([\d.]+)', browser_info)
            
            if browser_version:
                chromium_version = browser_version.group(1)
                print(f'Detected Chromium version: {chromium_version}')
                return chromium_version
        
        print('Failed to detect Chromium version from remote debugging')
        return None
        
    except Exception as e:
        print(f'Error detecting Chromium version: {e}')
        return None
    finally:
        # Clean up
        try:
            process.terminate()
            process.wait(timeout=5)
        except:
            process.kill()
        
        # Remove temp directory
        try:
            shutil.rmtree(temp_dir, ignore_errors=True)
        except:
            pass

In [3]:
MEMORYPC_COMMON_CONFIGURATION = {
    "case": "Fractal Design North XL Charcoal Black TG Dark Tint - Fenêtre en verre",
    "CPU": "AMD Ryzen 7 9800X3D, 8x 4.70Ghz",
    "motherboard": "MSI MAG B850 Tomahawk Max WIFI",
    "cooler": [
        "Arctic Liquid Freezer III Pro 360 A-RGB - 360mm - Noir",
        "MAG CORELIQUID A13 360, Noir - 360mm", # MSI x MPC  or MSI
        "be quiet! Pure Loop 3 LX - 360mm",
    ],
    "RAM": [
        "32Go DDR5 RAM 6000 MHz Kingston FURY Beast CL30 (2x 16Go - Dual Channel)",
        "32Go DDR5",
    ],
    "PSU": [
        "be quiet ! Pure Power 13 M - 1000W entièrement modulable - 80 PLUS Gold",
        "1000W",
        "1000 Watt",
    ],
    "SSD": [
        "1000 Go MSI Spatium M560 (Lecture : 10200MB/s | Ecriture : 8400MB/s)",
        "1000 Go WD Black SN7100 (Lecture : 7250MB/s | Ecriture : 6900MB/s)",
        "1000 Go",
    ],
    "GPU": "NVIDIA GeForce RTX 5080 - 16Go - changement de fabricant - compatible DLSS 4.5",
}

MEMORYPC_CONFIGURATIONS = {
    "RTX 5080": {
        **MEMORYPC_COMMON_CONFIGURATION,
    },
    "RTX 5070Ti": {
        **MEMORYPC_COMMON_CONFIGURATION,
        "GPU": "NVIDIA GeForce RTX 5070 Ti - 16Go - changement de fabricant - compatible DLSS 4.5"
    }
}


MEMORYPC_SEARCH_PAGES = [
    "https://www.memorypc.fr/pc-gamer-ordinateur-gaming/?limit=60",
    "https://www.memorypc.fr/pc-gamer-ordinateur-gaming/acheter-un-pc-gamer-amd/?limit=60",
    "https://www.memorypc.fr/pc-gamer-high-end/?limit=60",
    "https://www.memorypc.fr/editions-partenaires/?limit=60",
    "https://www.memorypc.fr/configurateur-pc/?limit=60",
    "https://www.memorypc.fr/ordinateur-de-bureau/?limit=60",
]

In [4]:
CSL_COMPUTER_COMMON_CONFIGURATION = {
    "case": "Fractal Design North XL, Charcoal Black TG Dark, panneau latéral en verre",
    "CPU": "AMD Ryzen 7 9800X3D, 8x 4700 MHz",
    "motherboard": "MSI MAG B850 Tomahawk Max WiFi",
    "cooler": [
        "MSI MAG CoreLiquid A13 360 refroidissement à eau",
        "be quiet ! Pure Loop 3 LX 360mm refroidissement par eau",
    ],
    "RAM": [
        "32 Go DDR5-RAM, double canal (2x 16 Go), 6000 MHz*, Kingston/G.Skill/Corsair/Patriot",
        "32 Go DDR5-RAM",
    ],
    "PSU": [
        "1000 Watt be quiet ! Pure Power 13 M, ATX3.1, 94% d'efficacité, certifié 80 Plus Gold",
        "1000 Watt",
        "1000W",
    ],
    "SSD": [
        "1000 Go M.2 PCIe 5.0 SSD Samsung 9100 Pro, lecture/écriture : max. 14700 Mo/s | 13300 Mo/s",
        "1000 Go M.2 PCIe 4.0 SSD Western Digital Black SN7100, lecture/écriture: max. 7250 Mo/s | 6900 Mo/s",
        "1000 Go M.2 PCIe 4.0 SSD",
    ],
    "GPU": [
        "ASUS GeForce RTX 5080, ASUS PRIME-RTX5080-16G-GAMING, 16 Go GDDR7, 1x HDMI, 3x DisplayPort, noir",
        "MSI GeForce RTX 5080, MSI GeForce RTX 5080 16G SHADOW 3X OC, 16 Go GDDR7, 1x HDMI, 3x DisplayPort, noir",
    ]
}

CSL_COMPUTER_CONFIGURATIONS = {
    "RTX 5080": {
        **CSL_COMPUTER_COMMON_CONFIGURATION
    },
    "RTX 5070Ti": {
        **CSL_COMPUTER_COMMON_CONFIGURATION,
        "GPU": [
            "MSI GeForce RTX 5070 Ti, MSI GeForce RTX 5070 Ti 16G SHADOW 3X OC, 16 Go GDDR7, 1x HDMI, 3x DisplayPort, noir",
            "ASUS GeForce RTX 5070 Ti, ASUS PRIME-RTX5070TI-16G, 16 Go GDDR7, 1x HDMI, 3x DisplayPort, noir"
        ]
    }
}

CSL_COMPUTER_SEARCH_PAGES = [
    "https://www.csl-computer.com/fr/pc-gamer/?limit=60",
    "https://www.csl-computer.com/fr/configurateur-pc/configurateur-pc-amd/?limit=60",
    "https://www.csl-computer.com/fr/configurateur-pc/configurateur-pc-intel/?limit=60",
    "https://www.csl-computer.com/fr/offres/?limit=60"
]


In [5]:
BRAVE_PATH = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser" # Update this path to your Brave browser location
BRAVE_USER_DATA_DIR = "/tmp/brave-selenium-profile"  # Update this path to your Brave user data directory
BRAVE_DEBUG_PORT = 9222  # Default port for Brave debugging
HEADLESS = True
NB_THREADS = 5
# Auto-detect the version or use manually defined value
BRAVE_VERSION = get_chromium_version_from_brave(BRAVE_PATH, BRAVE_DEBUG_PORT) or "144.0.7559.59"  # Fallback to manual version

Detected Chromium version: 144.0.7559.59


In [6]:
def get_chrome_options(headless=True):
    """Simplified Chrome options for speed and compatibility"""
    options = webdriver.ChromeOptions()
    options.binary_location = BRAVE_PATH
    
    # Essential arguments for stability
    essential_args = [
        '--no-sandbox',
        '--disable-dev-shm-usage',
        '--disable-blink-features=AutomationControlled',
        '--start-maximized',
        '--window-size=1920,1080',
        '--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'
    ]
    
    # Performance optimizations
    performance_args = [
        '--disable-extensions',
        '--disable-notifications',
        '--disable-popup-blocking',
        '--disable-default-apps',
        '--disable-sync',
        '--no-first-run',
        '--no-default-browser-check',
        '--disable-background-networking',
        '--disable-client-side-phishing-detection',
        '--disable-component-update',
        '--password-store=basic',
        '--use-mock-keychain',
    ]
    
    for arg in essential_args + performance_args:
        options.add_argument(arg)
    
    if headless:
        options.add_argument('--headless=new')
    
    # Stealth mode to bypass detection
    options.add_experimental_option("useAutomationExtension", False)
    options.add_experimental_option("excludeSwitches", ["enable-automation", "enable-logging"])
    
    # Optimized preferences
    prefs = {
        'profile.default_content_setting_values.notifications': 2,
        'download.prompt_for_download': False,
        'safebrowsing.enabled': False,
    }
    options.add_experimental_option("prefs", prefs)
    
    # Set timeouts - increased for DOM-heavy operations
    options.timeouts = {
        'script': 120000,  # 120 seconds for complex DOM manipulation
        'pageLoad': 60000,  # 60 seconds for page load
        'implicit': 0       # Keep at 0, use explicit waits instead
    }
    
    # Page load strategy
    options.page_load_strategy = 'normal'
    
    return options


def get_driver_fast(binary_path=BRAVE_PATH, port=BRAVE_DEBUG_PORT, user_data_dir=BRAVE_USER_DATA_DIR, stealth_mode=True, headless=True):
    """Optimized driver initialization with headless detection bypass"""
    
    options = get_chrome_options(headless=headless)
    options.add_argument(f'--remote-debugging-port={port}')
    options.add_argument(f'--user-data-dir={user_data_dir}')
    
    # Create service
    service = BraveService(
        ChromeDriverManager(
            driver_version=BRAVE_VERSION,
            chrome_type=ChromeType.BRAVE
        ).install()
    )
    
    # Create driver
    driver = BraveDriver(service=service, options=options)
    
    # Apply stealth JavaScript patches
    if stealth_mode:
        stealth_js = """
        // Overwrite navigator properties
        Object.defineProperty(navigator, 'webdriver', { get: () => undefined });
        Object.defineProperty(navigator, 'plugins', { get: () => [1, 2, 3, 4, 5] });
        Object.defineProperty(navigator, 'languages', { get: () => ['en-US', 'en'] });
        
        // Add chrome runtime
        window.chrome = { runtime: {} };
        
        // Mock hardware
        Object.defineProperty(navigator, 'hardwareConcurrency', { get: () => 8 });
        Object.defineProperty(navigator, 'deviceMemory', { get: () => 8 });
        """
        
        driver.execute_cdp_cmd('Page.addScriptToEvaluateOnNewDocument', {
            'source': stealth_js
        })
    
    driver.implicitly_wait(0)
    
    # Close extra tabs
    if len(driver.window_handles) > 1:
        main_window = driver.window_handles[0]
        for handle in driver.window_handles[1:]:
            driver.switch_to.window(handle)
            driver.close()
        driver.switch_to.window(main_window)
    
    return driver


# Alias for backward compatibility
def get_driver(*args, **kwargs):
    return get_driver_fast(*args, **kwargs)


# Start of Selenium processing

In [7]:
class ConfigurationResult:
    def __init__(self, url, price, configuration: Dict[str, Union[str, List[str]]], applied_configuration: Dict[str, str], missing_components: Dict[str, str], applied_indexes: Dict[str, int]):
        self.url = url
        self.price = price
        self.configuration = configuration
        self.applied_configuration = applied_configuration
        self.missing_components = missing_components
        self.applied_indexes = applied_indexes

In [8]:
def get_userscript(url: str) -> Optional[str]:
    """
    Load the userscript from the given URL.
    """
    response = requests.get(url)
    return response.text if response.status_code == 200 else None


# Load the common library once at startup
COMMON_LIB_URL = "https://raw.githubusercontent.com/guillaume-elambert/tools/master/UserScripts/PC-Configurator/pc-configurator-common.js"
COMMON_LIB_SCRIPT = get_userscript(COMMON_LIB_URL)

if not COMMON_LIB_SCRIPT:
    raise RuntimeError(f"Failed to load common library from {COMMON_LIB_URL}")

In [ ]:
class ManufacturerHandler(ABC):
    def __init__(self, manufacturer_name: str, userscript_url: str, config_userscript_variable: str, result_userscript_variable: str):
        self.manufacturer_name: str = manufacturer_name
        self.userscript_url: str = userscript_url
        self.userscript: Union[str, None] = get_userscript(userscript_url)
        self.config_userscript_variable: str = config_userscript_variable
        self.result_userscript_variable: str = result_userscript_variable
        # Pre-compile script template to avoid repeated string operations
        self._script_template = None

    def _get_script_template(self, userscript_content: str) -> str:
        """Cache the script template to avoid rebuilding it every time"""
        if self._script_template is None:
            # Inject common library first, then the specific userscript
            self._script_template = f"""
                {self.config_userscript_variable} = arguments[0];
                {COMMON_LIB_SCRIPT}
                {userscript_content}
                await new Promise(resolve => setTimeout(resolve, 200));
                return {self.result_userscript_variable};
            """
        return self._script_template

    def handle_pc_page_configuration(self, configuration_name: str, configuration: Dict[str, Union[str, List[str]]], driver: WebDriver, url: str, userscript_content: Optional[str] = None) -> Optional[ConfigurationResult]:
        userscript_content = userscript_content or self.userscript or get_userscript(self.userscript_url)
        if not userscript_content:
            raise ValueError("Userscript content is not available.")
        
        driver.get(url)
        
        # Wait for page to be fully loaded and interactive
        driver.execute_script("""
            return new Promise((resolve) => {
                if (document.readyState === 'complete') {
                    resolve();
                } else {
                    window.addEventListener('load', resolve);
                }
            });
        """)
        
        def clear_storage():
            # Clear storage via CDP (much faster than execute_script)
            try:
                driver.execute_cdp_cmd('Storage.clearDataForOrigin', {
                    'origin': driver.execute_script('return location.origin'),
                    'storageTypes': 'all'
                })
            except:
                # Fallback to script-based clearing if CDP fails
                driver.execute_script("""
                    window.localStorage.clear();
                    window.sessionStorage.clear();
                """)

        clear_storage()
        
        # Small delay to ensure DOM is stable (adjust if needed: 100-500ms)
        # driver.execute_script("return new Promise(resolve => setTimeout(resolve, 200));")
        
        # Execute userscript and config in a single script call
        applied = driver.execute_script(self._get_script_template(userscript_content), configuration)
        # applied = driver.execute_script(self._get_result_template())

        clear_storage()

        if not applied:
            print(f"{url:<90} - {configuration_name:<30} - No configuration components found.")
            return None

        # Transform the JSON applied result into a ConfigurationResult object
        result = ConfigurationResult(
            url=url,
            price=applied.get("price", 0),
            configuration=configuration,
            applied_configuration=applied.get("applied_configuration", {}),
            missing_components=applied.get("missing_components", {}),
            applied_indexes=applied.get("applied_indexes", {})
        )
        

        if len(result.missing_components) > 0:
            print(f"{url:<90} - {configuration_name:<30} - Not all configuration components were found. Missing: {', '.join(result.missing_components)}")
            return result

        print(f"{url:<90} - {configuration_name:<30} - All configuration components found: {result.price}€")

        ordinal_suffix = {"1": "st", "2": "nd", "3": "rd"}
        
        # Build backup components message (optimized)
        if result.applied_indexes:
            backup_msgs = [
                f'{comp}: {idx+1}{ordinal_suffix.get(str(idx+1)[-1], "th")} component used ("{result.applied_configuration[comp]}")'
                for comp, idx in result.applied_indexes.items() if idx != 0
            ]
            
            if backup_msgs:
                print("Backup components used:\n" + "\n".join(backup_msgs))
    
        return result

    def handle_pc_page(self, configurations: Dict[str, Dict[str, Union[str, List[str]]]], driver: WebDriver, url: str, userscript_content: Optional[str] = None) -> Dict[str, ConfigurationResult]:
        """
        Handle the PC page, apply the configuration and return the result.
        Optimized version with proper waits for DOM-dependent scripts.
        """
        userscript_content = userscript_content or self.userscript or get_userscript(self.userscript_url)
        if not userscript_content:
            raise ValueError("Userscript content is not available.")

        results = {}
        
        for configuration_name, configuration in configurations.items():
            try:
                res = self.handle_pc_page_configuration(configuration_name, configuration, driver, url, userscript_content)
                # Retry once because it might be a JS script error
                if not res:
                    res = self.handle_pc_page_configuration(configuration_name, configuration, driver, url, userscript_content)
                
                if not res:
                    continue
                results[configuration_name] = res
            except:
                try:
                    # Retry once in case of transient errors
                    res = self.handle_pc_page_configuration(configuration_name, configuration, driver, url, userscript_content)
                    if res:
                        results[configuration_name] = res
                except Exception as e:
                    print(f"Error applying configuration {configuration_name} on {url}: {e}")

        return results

    @abstractmethod
    def find_search_page_configurations_url(self, driver: WebDriver) -> List[str]:
        pass

    @abstractmethod
    def find_next_search_page_url(self, driver: WebDriver) -> Optional[str]:
        pass

    def handle_search_page(self, configurations: Dict[str, Dict[str, Union[str, List[str]]]], driver: WebDriver, url: str, nb_threads: int = 10, headless: bool = False) -> List[Dict[str, ConfigurationResult]]:
        """
        Optimized version using ThreadPoolExecutor and better resource management.
        """
        results: List[Dict[str, ConfigurationResult]] = []
        userscript_content = self.userscript or get_userscript(self.userscript_url)

        if not userscript_content:
            print("Failed to load userscript. Please check the URL.")
            return results
        
        # Prepare thread resources
        user_data_dirs = [f"{BRAVE_USER_DATA_DIR}_thread_{self.manufacturer_name.lower().strip()}_{i+1}" for i in range(nb_threads)]
        
        # Use context manager for drivers
        @contextmanager
        def get_drivers():
            drivers = []
            try:
                # Create all drivers in parallel to speed up initialization
                with ThreadPoolExecutor(max_workers=nb_threads) as executor:
                    futures = [
                        executor.submit(get_driver_fast, port=BRAVE_DEBUG_PORT + i + 1, user_data_dir=user_data_dirs[i], stealth_mode=True, headless=headless)
                        for i in range(nb_threads)
                    ]
                    drivers = [f.result() for f in futures]
                yield drivers
            finally:
                # Close drivers in parallel
                with ThreadPoolExecutor(max_workers=nb_threads) as executor:
                    for d in drivers:
                        executor.submit(lambda driver: driver.quit(), d)
                
                # Cleanup data directories in parallel
                with ThreadPoolExecutor(max_workers=nb_threads) as executor:
                    for user_data_dir in user_data_dirs:
                        executor.submit(shutil.rmtree, user_data_dir, ignore_errors=True)
        
        with get_drivers() as thread_webdrivers:
            search_page_url = url
            
            while search_page_url:
                driver.get(search_page_url)
                search_page_tab = driver.current_window_handle

                # Get all configuration URLs on the search page
                configurations_urls = self.find_search_page_configurations_url(driver)
                print(f"Navigating to search page: {search_page_url}. Found {len(configurations_urls)} configurations.")
                
                # Use ThreadPoolExecutor for better thread management
                with ThreadPoolExecutor(max_workers=nb_threads) as executor:
                    # Submit all tasks at once
                    future_to_url = {}
                    for i, config_url in enumerate(configurations_urls):
                        thread_driver = thread_webdrivers[i % nb_threads]
                        future = executor.submit(
                            self.handle_pc_page,
                            configurations,
                            thread_driver,
                            config_url,
                            userscript_content
                        )
                        future_to_url[future] = config_url
                    
                    # Collect results as they complete
                    for future in as_completed(future_to_url):
                        try:
                            result = future.result(timeout=90)  # Match script timeout
                            if result:
                                results.append(result)
                        except Exception as e:
                            print(f"Error processing {future_to_url[future]}: {e}")
                
                print(f"Finished processing: {len(results)} configurations stored.")
                    
                # Select the search page tab
                driver.switch_to.window(search_page_tab)

                # Find the next search page URL
                search_page_url = self.find_next_search_page_url(driver)
        
        # Close all other tabs except the original one
        for handle in driver.window_handles:
            if handle != search_page_tab:
                driver.switch_to.window(handle)
                driver.close()
        
        return results

In [10]:
class MemoryPCHandler(ManufacturerHandler):

    def __init__(self):
        super().__init__(
            manufacturer_name="MemoryPC",
            userscript_url="https://raw.githubusercontent.com/guillaume-elambert/tools/master/UserScripts/PC-Configurator/memorypc-automatic-configuration.js",
            config_userscript_variable="window.MEMORYPC_CONFIG",
            result_userscript_variable="window.MEMORYPC_CONFIG_RESULT"
        )
        
    def find_search_page_configurations_url(self, driver: CustomChromeWebDriver) -> List[str]:    
        items_buttons: list[WebElement] = driver.find_elements(By.CSS_SELECTOR, ".product-info > .product-action > .pdp-box-configurator > .btn-configurator")
        buttons_links: list[Union[str, None]] = [button.get_attribute("href") for button in items_buttons]
        return [link for link in buttons_links if link is not None]
    
    def find_next_search_page_url(self, driver: CustomChromeWebDriver) -> Optional[str]:
        # Select .pagination-nav .page-item.page-next:not(.disabled)
            try:
                next_page = driver.find_element(By.CSS_SELECTOR, "nav.pagination-nav > .pagination > .page-item.page-next:not(.disabled) > a.page-link")
            except Exception as e:
                print("No more pages to navigate.")
                return None
            
            return next_page.get_attribute("href") if next_page else None

In [11]:
class CSLComputerHandler(ManufacturerHandler):
    
    def __init__(self):
        super().__init__(
            manufacturer_name="CSL Computer",
            userscript_url="https://raw.githubusercontent.com/guillaume-elambert/tools/master/UserScripts/PC-Configurator/csl-computer-automatic-configuration.js",
            config_userscript_variable="window.CSL_COMPUTER_CONFIG",
            result_userscript_variable="window.CSL_COMPUTER_CONFIG_RESULT"
        )
        
    def find_search_page_configurations_url(self, driver: WebDriver) -> List[str]:    
        items_buttons: list[WebElement] = driver.find_elements(By.CSS_SELECTOR, ".products-list article.product a.product-link")
        buttons_links: list[Union[str, None]] = [button.get_attribute("href") for button in items_buttons]
        return [link for link in buttons_links if link is not None]
    
    def find_next_search_page_url(self, driver: WebDriver) -> Optional[str]:
        # Select .pagination-nav .page-item.page-next:not(.disabled)
            try:
                next_page = driver.find_element(By.CSS_SELECTOR, ".toolbar.bottom > .pages > .pagination .next:not(.off)")
            except Exception as e:
                print("No more pages to navigate.")
                return None
            
            return next_page.get_attribute("href") if next_page else None

In [12]:
driver = get_driver(headless=HEADLESS)
driver.maximize_window()
memorypc_results: list[Dict[str, ConfigurationResult]] = []

handler = MemoryPCHandler()
with keep.presenting():
    for search_page in MEMORYPC_SEARCH_PAGES:
        try:
            memorypc_results.extend(handler.handle_search_page(configurations=MEMORYPC_CONFIGURATIONS, driver=driver, url=search_page, nb_threads=NB_THREADS, headless=HEADLESS))
        except Exception as e:
            print(f"Error processing {search_page}: {e}")

driver.close()
driver.quit()
#kill_brave_process()


Navigating to search page: https://www.memorypc.fr/pc-gamer-ordinateur-gaming/?limit=60. Found 60 configurations.
https://www.memorypc.fr/pc-gamer/amd-ryzen-5-557636/                                       - RTX 5080                       - Not all configuration components were found. Missing: CPU, RAM, motherboard
https://www.memorypc.fr/pc-gamer/amd-ryzen-5-560629/                                       - RTX 5080                       - All configuration components found: 3109.21€
Backup components used:
cooler: 3rd component used ("be quiet! Pure Loop 3 LX - 360mm")
https://www.memorypc.fr/pc-gamer/amd-ryzen-7-557784/                                       - RTX 5080                       - Not all configuration components were found. Missing: CPU, RAM, motherboard
https://www.memorypc.fr/pc-gamer/amd-ryzen-7-554088/                                       - RTX 5080                       - Not all configuration components were found. Missing: CPU, RAM, motherboard
https://www.memorypc.

In [13]:
driver = get_driver(headless=HEADLESS)
driver.maximize_window()
csl_computer_results: list[Dict[str, ConfigurationResult]] = []

handler = CSLComputerHandler()
with keep.presenting():
    for search_page in CSL_COMPUTER_SEARCH_PAGES:
        try:
            csl_computer_results.extend(handler.handle_search_page(configurations=CSL_COMPUTER_CONFIGURATIONS, driver=driver, url=search_page, nb_threads=NB_THREADS, headless=HEADLESS))
        except Exception as e:
            print(f"Error processing {search_page}: {e}")

driver.close()
driver.quit()
#kill_brave_process()

Navigating to search page: https://www.csl-computer.com/fr/pc-gamer/?limit=60. Found 60 configurations.
https://www.csl-computer.com/fr/pc-gamer/pc-csl-sprint-5756-ryzen-7-9326.html              - RTX 5080                       - No configuration components found.
https://www.csl-computer.com/fr/pc-gamer/pc-csl-sprint-5522-ryzen-5.html                   - RTX 5080                       - No configuration components found.
https://www.csl-computer.com/fr/pc-gamer/pc-csl-sprint-5682-ryzen-5.html                   - RTX 5080                       - No configuration components found.
https://www.csl-computer.com/fr/pc-gamer/pc-csl-sprint-5683-ryzen-5.html                   - RTX 5080                       - No configuration components found.
https://www.csl-computer.com/fr/pc-gamer/pc-csl-sprint-5679-ryzen-5.html                   - RTX 5080                       - No configuration components found.
https://www.csl-computer.com/fr/pc-gamer/pc-csl-sprint-5522-ryzen-5.html                   

In [14]:
def compute_results_per_configuration(results: list[Dict[str, ConfigurationResult]]) -> Dict[str, List[ConfigurationResult]]:
    results_per_configuration: Dict[str, List[ConfigurationResult]] = {}
    for result_item in results:
        for configuration_name, result in result_item.items():
            if configuration_name not in results_per_configuration:
                results_per_configuration[configuration_name] = []
            results_per_configuration[configuration_name].append(result)
    return results_per_configuration

# memorypc_results_per_configuration: Dict[str, List[ConfigurationResult]] = {}
# for result_item in memorypc_results:
#     for configuration_name, result in result_item.items():
#         if configuration_name not in memorypc_results_per_configuration:
#             memorypc_results_per_configuration[configuration_name] = []
#         memorypc_results_per_configuration[configuration_name].append(result)

In [15]:
# csl_computer_results_per_configuration: Dict[str, List[ConfigurationResult]] = {}
# for result_item in csl_computer_results:
#     for configuration_name, result in result_item.items():
#         if configuration_name not in csl_computer_results_per_configuration:
#             csl_computer_results_per_configuration[configuration_name] = []
#         csl_computer_results_per_configuration[configuration_name].append(result)
memorypc_results_per_configuration = compute_results_per_configuration(memorypc_results)
csl_computer_results_per_configuration = compute_results_per_configuration(csl_computer_results)

In [16]:
def print_results(results_per_configuration: Dict[str, List[ConfigurationResult]]):
    index = 0
    for configuration_name, config_results in results_per_configuration.items():
        index += 1
        
        if(index>1):
            print("\n\n\n\n")
        print(f"===================== {configuration_name} ====================")
        
        # Remove duplicates based on URL, keeping the one with the most components applied and the lowest price
        unique_results: list[ConfigurationResult] = []
        for result in config_results:
            if not any(r.url == result.url for r in unique_results):
                unique_results.append(result)
            else:
                existing_result = next(r for r in unique_results if r.url == result.url)
                if (len(result.missing_components) < len(existing_result.missing_components)): 
                    unique_results.remove(existing_result)
                    unique_results.append(result)
                    continue
                
                if (result.price < existing_result.price):
                    unique_results.remove(existing_result)
                    unique_results.append(result)

        # Sort on both nb elements missing and price
        sorted_results: List[ConfigurationResult] = sorted(config_results, key=lambda r: (len(set(r.configuration.keys()) - set(r.applied_configuration.keys())), r.price))

        # Print the results
        print(f"Results ({len(sorted_results)}):")
        result_strings = []

        for result in sorted_results:
            # if len(result.missing_components) > 0:
            #     continue
            missing_components_str = "None"
            if len(result.missing_components) > 0:
                missing_components_str = str(len(result.missing_components))
                missing_components_str += ": " + ', '.join(result.missing_components)
            
            backup_components_str = "None"
            backup_components_strings = []
            for component, index in result.applied_indexes.items():
                if index != 0:
                    th = { "1": "st", "2": "nd", "3": "rd" }.get(str(index+1)[-1], "th")
                    backup_components_strings.append(f'{component}: {str(index+1)}{th}')
            
            if backup_components_strings:
                backup_components_str = ', '.join(backup_components_strings)
                
            result_strings.append([
                # In price, replace '.' with ',' for better readability
                f"{result.price:.2f} €".replace('.', ','),
                f"{missing_components_str}",
                f"{backup_components_str}",
                f"{result.url}"
            ])
            
        # Add at the header at the beginning the result_strings
        result_strings.insert(0, [
            "Price",
            "Missing components",
            "Backup components",
            "URL"
        ])
        max_lengths = [len(col) for col in result_strings[0]]

        # Get the maximum length of each column
        for row in result_strings:
            for index, column in enumerate(row):
                if len(column) > max_lengths[index]:
                    max_lengths[index] = len(column)


        # Print the results
        for row in result_strings:
            print("\t|\t".join(f"{column:<{max_lengths[index]}}" for index, column in enumerate(row)))
  
        

In [17]:
print_results(memorypc_results_per_configuration)

===================== RTX 5080 ====================
Results (283):
Price    	|	Missing components                    	|	Backup components    	|	URL                                                                               
3038,62 €	|	None                                  	|	cooler: 3rd          	|	https://www.memorypc.fr/pc-gamer/amd-ryzen-5-557548/                              
3038,62 €	|	None                                  	|	cooler: 3rd          	|	https://www.memorypc.fr/pc-gamer/amd-ryzen-5-557548/                              
3078,96 €	|	None                                  	|	cooler: 3rd          	|	https://www.memorypc.fr/pc-gamer/amd-ryzen-5-559111/                              
3078,96 €	|	None                                  	|	cooler: 3rd          	|	https://www.memorypc.fr/pc-gamer/amd-ryzen-5-559111/                              
3089,04 €	|	None                                  	|	cooler: 3rd          	|	https://www.memorypc.fr/pc-gamer/amd-ryzen-7-557287/    

In [18]:
print_results(csl_computer_results_per_configuration)

===================== RTX 5080 ====================
Results (76):
Price    	|	Missing components                                   	|	Backup components	|	URL                                                                                                                                        
2999,00 €	|	7: CPU, PSU, RAM, SSD, case, cooler, motherboard     	|	GPU: 2nd         	|	https://www.csl-computer.com/fr/pc-gamer/pc-csl-sprint-5952-ryzen-9.html                                                                   
429,00 € 	|	8: CPU, GPU, PSU, RAM, SSD, case, cooler, motherboard	|	None             	|	https://www.csl-computer.com/fr/offres/mini-pc-csl-narrow-box-core-i3-windows-11-home-500gb-16gb.html                                      
469,00 € 	|	8: CPU, GPU, PSU, RAM, SSD, case, cooler, motherboard	|	None             	|	https://www.csl-computer.com/fr/configurateur-pc/configurateur-pc-intel/csl-gaming-pc-konfigurator-intel-12-13-14-gen-ddr5-sockel-1700.html
779,00 € 	|	8: CPU, GP

In [19]:
import json

print("=========== MEMORYPC ===========")
for configuration_name, configuration in MEMORYPC_CONFIGURATIONS.items():
    print(f"Configuration: {configuration_name}")
    print(json.dumps(configuration, indent=4, ensure_ascii=False))
    print("\n\n")

print("\n\n=========== CSL COMPUTER ===========")
for configuration_name, configuration in CSL_COMPUTER_CONFIGURATIONS.items():
    print(f"Configuration: {configuration_name}")
    print(json.dumps(configuration, indent=4, ensure_ascii=False))
    print("\n\n")
    

=========== MEMORYPC ===========
Configuration: RTX 5080
{
    "case": "Fractal Design North XL Charcoal Black TG Dark Tint - Fenêtre en verre",
    "CPU": "AMD Ryzen 7 9800X3D, 8x 4.70Ghz",
    "motherboard": "MSI MAG B850 Tomahawk Max WIFI",
    "cooler": [
        "Arctic Liquid Freezer III Pro 360 A-RGB - 360mm - Noir",
        "MAG CORELIQUID A13 360, Noir - 360mm",
        "be quiet! Pure Loop 3 LX - 360mm"
    ],
    "RAM": [
        "32Go DDR5 RAM 6000 MHz Kingston FURY Beast CL30 (2x 16Go - Dual Channel)",
        "32Go DDR5"
    ],
    "PSU": [
        "be quiet ! Pure Power 13 M - 1000W entièrement modulable - 80 PLUS Gold",
        "1000W",
        "1000 Watt"
    ],
    "SSD": [
        "1000 Go MSI Spatium M560 (Lecture : 10200MB/s | Ecriture : 8400MB/s)",
        "1000 Go WD Black SN7100 (Lecture : 7250MB/s | Ecriture : 6900MB/s)",
        "1000 Go"
    ],
    "GPU": "NVIDIA GeForce RTX 5080 - 16Go - changement de fabricant - compatible DLSS 4.5"
}



Configuration: RTX 

In [20]:
driver = get_driver(headless=True)
driver.maximize_window()
csl_computer_results: list[Dict[str, ConfigurationResult]] = []

handler = CSLComputerHandler()
res = handler.handle_pc_page(configurations=CSL_COMPUTER_CONFIGURATIONS, driver=driver, url="https://www.csl-computer.com/fr/pc-gamer/pc-csl-sprint-5812-ryzen-7.html")

driver.close()
driver.quit()
#kill_brave_process()

https://www.csl-computer.com/fr/pc-gamer/pc-csl-sprint-5812-ryzen-7.html                   - RTX 5080                       - No configuration components found.
https://www.csl-computer.com/fr/pc-gamer/pc-csl-sprint-5812-ryzen-7.html                   - RTX 5080                       - Not all configuration components were found. Missing: CPU, GPU, PSU, RAM, SSD, case, cooler, motherboard
https://www.csl-computer.com/fr/pc-gamer/pc-csl-sprint-5812-ryzen-7.html                   - RTX 5070Ti                     - No configuration components found.
https://www.csl-computer.com/fr/pc-gamer/pc-csl-sprint-5812-ryzen-7.html                   - RTX 5070Ti                     - No configuration components found.
